In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential


In [ ]:
# re-size all the images to this
SIZE = 240

train_path = '/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition-20221108T101958Z-001/Masked Face Recognition/MainDataset'
# valid_path = '/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/database/Test'


In [ ]:
# Training data and labels into respective lists
train_images = []
train_labels = []
training_data = []
prev=0

for directory_path in glob.glob(os.path.join(train_path, "*")):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        # print(img_path)
        img = cv2.imread(img_path)       
        img = cv2.resize(img, (SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        training_data.append([train_images,train_labels])
    print(len(train_images)-prev)
    prev=len(train_images)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
training_data = np.array(training_data)

In [ ]:
training_data = np.array(training_data,dtype=object)
len(train_labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=45)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
train_labels_encoded = encoder.transform(train_labels)
encoder.fit(test_labels)
test_labels_encoded = encoder.transform(test_labels)

In [ ]:
#Inverse transform to get original label back. 
for i in range (0,12):
  print(i," ",encoder.inverse_transform([i]))

In [ ]:
train_images = np.array(train_images).reshape(-1,SIZE,SIZE,3)
train_images=np.array(train_images)
train_images.shape

In [ ]:
train_labels=np.array(train_labels)
train_labels.shape

In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
print('x_train       ',len(x_train),"         ",x_train.shape)
print('y_train       ',len(y_train),"         ",y_train.shape)
print('x_test       ',len(x_test),"         ",x_test.shape)
print('y_test       ',len(y_test),"         ",y_test.shape)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        brightness_range=[0.5,1.0],
                        horizontal_flip=True)

In [ ]:
len(os.listdir(train_path))

In [ ]:
# Loading model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
flatten = Flatten()(vgg16.output)
dense1 = Dense(192,activation='relu')(flatten)
dense2 = Dense(48,activation='relu')(dense1)
# dense3 = Dense(24,activation='relu')(dense2)
prediction = Dense(len(os.listdir(train_path)), activation='softmax')(dense2)

model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=20,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
result = model.fit(data_generator.flow(x_train,y_train,batch_size=16),
                   validation_data=(x_test,y_test),
                   epochs = 150, callbacks = [callback])

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition-20221108T101958Z-001/Masked Face Recognition/5model_vgg16.h5')

In [ ]:
model1=model

In [ ]:
from tensorflow.keras.models import load_model
model1=load_model('/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition-20221108T101958Z-001/Masked Face Recognition/5model_vgg16.h5')

In [ ]:
y_pred = model1.predict(test_images)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
y_pred = encoder.inverse_transform(y_pred)
y_pred

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, y_pred))

In [ ]:
#Confusion Matrix - accuracy of each class
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels, y_pred)
sns.heatmap(cm, annot=True)

In [ ]:
# For selected images
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
# img = 
plt.imshow(img)
input_img = np.expand_dims(img, axis=0)
pred=model1.predict(input_img)
pred = np.argmax(pred, axis=1)
pred = encoder.inverse_transform([pred])
print("The prediction for this image is: ", pred)
print("The actual label for this image is: ", test_labels[n])